In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

## Import relevant libraries

In [ ]:
# core system imports
import os

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import string
import random
import joblib
import itertools

from timeit import timeit
from unidecode import unidecode
import matplotlib.pyplot as plt

from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Dense, 
    Input, 
    LSTM, 
    Embedding, 
    Dropout, 
    GlobalMaxPool1D
)
from tensorflow.keras.models import Sequential

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import (
    validation_curve,
    learning_curve
)
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
    precision_recall_fscore_support
)

# Matplotlib config
%matplotlib inline
%alias_magic t timeit


C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Python37\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Python37\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: A dynamic link library (DLL) initialization routine failed.

During handling of the

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Python37\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Python37\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: A dynamic link library (DLL) initialization routine failed.

During handling of the

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Python37\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Python37\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: A dynamic link library (DLL) initialization routine failed.

During handling of the

In [ ]:
# check for available GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

ERROR! Session/line number was not unique in database. History logging moved to new session 383


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3594535560.py", line 2, in <module>
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3594535560.py", line 2, in <module>
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\cor

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3594535560.py", line 2, in <module>
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\cor

In [2]:
# Location of the pipeline metadata store
_pipeline_root = '../pipeline/'

# Directory of the raw data files
_data_root = '../input/'

# Directory of the pretrained word embeddings
_embedding_root = '../input/embeddings'

_data_filepath = os.path.join(_data_root, "corona_dataset_1.csv")

_stopwords_filepath = os.path.join(_data_root, "stopwords.txt")

_embedding_model_filepath = os.path.join(_embedding_root, "model_sg")
_pretrained_vectors_filepath = os.path.join(_embedding_root, "model_sg.wv.vectors.npy")
_neg_vectors_filepath = os.path.join(_embedding_root, "model_sg.trainables.syn1neg.npy")


In [3]:
# List input datasets in directory
os.listdir(_data_root)

# Word embeddings
print(os.listdir(_embedding_root))

ERROR! Session/line number was not unique in database. History logging moved to new session 386
['model_sg', 'model_sg.trainables.syn1neg.npy', 'model_sg.wv.vectors.npy']


## Load training and test data

In [4]:
# Read data from CSV file
data = pd.read_csv(_data_filepath, encoding="ISO-8859-1")

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   text       1497 non-null   object 
 1   author_id  1497 non-null   float64
 2   Label      1497 non-null   object 
dtypes: float64(1), object(2)
memory usage: 35.2+ KB


In [6]:
data.head()

,text,author_id,Label
0,- @aishambuhari ta nemi babban sufeton 'yan sa...,7.970000e+17,neutral
1,"""Duk dan Bokon da baida Ilimin Addini Annoba n...",2.290470e+09,neutral
2,"""Duk mutumin da yayi tunanin bawa mutane ilimi...",1.071387e+09,neutral
3,"""Duk wanda ya sabawa dokar kaucewa kamuwa daga...",1.260000e+18,Positive
4,"""Duk wanda ya san ya fito daga yankin da ake A...",1.039268e+09,Positive


In [7]:
# Read stop words
stopwords_list = list()

with open(_stopwords_filepath) as file:
    stopwords_list = [line.strip() for line in file.readlines()]

In [8]:
listToStr = ' '.join([str(word) for word in stopwords_list])
print(listToStr)

ta da ya sai ba yi na kuma ma ji cikin in ni wata wani ce tana don za sun amma ga ina ne mai suka wannan a ko lokacin su take shi yake yana ka ban ita tafi


In [9]:
# Read saved data from disk
def load_pickle(filename):
    data = joblib.load(filename)
    return(data)
    
# Save data to disk for future use
def save_pickle(data, filename):
    joblib.dump(data, filename)
    return True

In [10]:
# removing stopwords
def _apply_lowercase(text):
    text = [item for item in text if item not in stopwords_list]
    text = ''.join(text)
    return text

# removing stopwords
def _stopwords_removal(text):
    text = [item for item in text if item not in stopwords_list]
    text = ''.join(text)
    return text

# remove punctuations
def _punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str

# Shuffle dataset
def _shuffle_dataset(dataset):
    dataset = shuffle(data)
    dataset = data.reset_index(drop=True)
    return dataset

In [ ]:
# Change string to lower case
data['text'] = data['text'].apply(str.lower)

# Change label to lower case
data['Label'] = data['Label'].apply(str.lower)

# remove punctuations or special characters
data['text'] = data['text'].apply(_punctuation_removal)

# remove stopwords
data['text'] = data['text'].apply(_stopwords_removal)

# Shuffle the dataset to prevent bias:
data = _shuffle_dataset(data)

# Print head of the data
data.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3773713990.py", line 8, in <module>
    data['text'] = data['text'].apply(_punctuation_removal)
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\pandas\core\series.py", line 4213, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas\_libs\lib.pyx", line 2403, in pandas._libs.lib.map_infer
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/1813181653.py", line 15, in _punctuation_removal
    all_list = [char for char in text if char not in string.punctuation]
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/1813181653.py", line 15, in <listcomp>
    all_list = [char for char in text if char not in string.punctuation]
NameErr

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3773713990.py", line 8, in <module>
    data['text'] = data['text'].apply(_punctuation_removal)
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\pandas\core\series.py", line 4213, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas\_libs\lib.pyx", line 2403, in pandas._libs.lib.map_infer
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/1813181653.py", line 15, in _punctuation_removal
    all_list = [char for char in text if char not in string.punctuation]
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/1813181653.py", line 15, in <listcomp>
    all_list = [char for char in text if char not in string.punctuation]
NameErr

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3773713990.py", line 8, in <module>
    data['text'] = data['text'].apply(_punctuation_removal)
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\pandas\core\series.py", line 4213, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas\_libs\lib.pyx", line 2403, in pandas._libs.lib.map_infer
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/1813181653.py", line 15, in _punctuation_removal
    all_list = [char for char in text if char not in string.punctuation]
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/1813181653.py", line 15, in <listcomp>
    all_list = [char for char in text if char not in string.punctuation]
NameErr

### Encode output lalbel

In [ ]:
# make copy of dataframe
data_encoded = data.copy()

# create a label encoder
label_encoder = LabelEncoder()
label_encoder.fit(data_encoded['Label'])

data_encoded['label_encoded'] = label_encoder.transform(data_encoded['Label'])

data_encoded.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/761800930.py", line 5, in <module>
    label_encoder = LabelEncoder()
NameError: name 'LabelEncoder' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 387
Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/761800930.py", line 5, in <module>
    label_encoder = LabelEncoder()
NameError: name 'LabelEncoder' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_a

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/761800930.py", line 5, in <module>
    label_encoder = LabelEncoder()
NameError: name 'LabelEncoder' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_no

### Split Dataset

+ Tranining: 70% of the dataset
+ Testing: 30% of dataset

In [ ]:
list_classes = ["label_encoded"]

X_train, X_test, Y_train, Y_test = train_test_split(data_encoded["text"], data_encoded[list_classes], test_size=0.3, random_state = 1)

Y_train = Y_train.values.flatten()
Y_test = Y_test.values.flatten()

# Show dimension of the comments
X_train.shape, X_test.shape

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 390
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/478781599.py", line 3, in <module>
    X_train, X_test, Y_train, Y_test = train_test_split(data_encoded["text"], data_encoded[list_classes], test_size=0.3, random_state = 1)
NameError: name 'train_test_split' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    st

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/478781599.py", line 3, in <module>
    X_train, X_test, Y_train, Y_test = train_test_split(data_encoded["text"], data_encoded[list_classes], test_size=0.3, random_state = 1)
NameError: name 'train_test_split' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/478781599.py", line 3, in <module>
    X_train, X_test, Y_train, Y_test = train_test_split(data_encoded["text"], data_encoded[list_classes], test_size=0.3, random_state = 1)
NameError: name 'train_test_split' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS

### Tokenize Sentences

To be able to train our model with a text data, we'd have to convert it into number form, for this we're going to use the Tokenizer module from Keras.preprocessing library

In [ ]:
list_sentences_train = X_train.values
list_sentences_test = X_test.values

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/405933530.py", line 1, in <module>
    list_sentences_train = X_train.values
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/405933530.py", line 1, in <module>
    list_sentences_train = X_train.values
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/405933530.py", line 1, in <module>
    list_sentences_train = X_train.values
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_

In [ ]:
tokenizer = Tokenizer(lower=True)

tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/2212784077.py", line 1, in <module>
    tokenizer = Tokenizer(lower=True)
NameError: name 'Tokenizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/2212784077.py", line 1, in <module>
    tokenizer = Tokenizer(lower=True)
NameError: name 'Tokenizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_n

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/2212784077.py", line 1, in <module>
    tokenizer = Tokenizer(lower=True)
NameError: name 'Tokenizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_ast_n

In [ ]:
# size of training vocabulary (number of unique words)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3445852849.py", line 2, in <module>
    vocab_size = len(tokenizer.word_index) + 1
NameError: name 'tokenizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", lin

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3445852849.py", line 2, in <module>
    vocab_size = len(tokenizer.word_index) + 1
NameError: name 'tokenizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/3445852849.py", line 2, in <module>
    vocab_size = len(tokenizer.word_index) + 1
NameError: name 'tokenizer' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in 

In [ ]:
# print out a random sequence of text from the tokenized training data
print(random.choice(list_tokenized_train))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/556707931.py", line 2, in <module>
    print(random.choice(list_tokenized_train))
NameError: name 'random' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/556707931.py", line 2, in <module>
    print(random.choice(list_tokenized_train))
NameError: name 'random' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/556707931.py", line 2, in <module>
    print(random.choice(list_tokenized_train))
NameError: name 'random' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3377, in run_

In [ ]:
print(len(list_tokenized_train), 'train sequences')
print(len(list_tokenized_test), 'test sequences')

print('Average train sequence length: {}'.format(np.mean(list(map(len, list_tokenized_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, list_tokenized_test)), dtype=int)))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/732884028.py", line 1, in <module>
    print(len(list_tokenized_train), 'train sequences')
NameError: name 'list_tokenized_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywrap_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/732884028.py", line 1, in <module>
    print(len(list_tokenized_train), 'train sequences')
NameError: name 'list_tokenized_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/732884028.py", line 1, in <module>
    print(len(list_tokenized_train), 'train sequences')
NameError: name 'list_tokenized_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.

## Pad tokenized Sentences

You might have observed that the sentences are not of the same lengths, so we need to pad them with zeros (0's) so that the resulting array will have equal length.

We'd use `text` module from `keras.preprocessing` library

We'd use a max character of 300

In [ ]:
maxlen = 300
X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/4138631341.py", line 2, in <module>
    X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
NameError: name 'pad_sequences' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\tensorflow_core\python\pywr

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/4138631341.py", line 2, in <module>
    X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
NameError: name 'pad_sequences' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshe

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SATELL~1\AppData\Local\Temp/ipykernel_12164/4138631341.py", line 2, in <module>
    X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
NameError: name 'pad_sequences' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Satellite\DS\sentiment_analysis_hausa\venv\lib\site-packages\IPython\core\interactiveshe

In [ ]:
print(X_train[:1])

## Use pre-trained Word Embedding

In [ ]:
# load saved genism model from disk
model_sg = Word2Vec.load(_embedding_model_filepath)

# load saved numpy vectors from disk
vectors = np.load(_pretrained_vectors_filepath)

# load a saved trainable numpy vectors from disk
vectors_neg = np.load(_neg_vectors_filepath)

In [ ]:
model_sg

In [ ]:
# get vocabularies from saved model
vocabs = list()

for word, vocab_obj in model_sg.wv.vocab.items():
    vocabs.append(word)

# delete unused variables to free memory
del((word, vocab_obj))

In [ ]:
# Check length of vectors
len(vectors), len(vectors_neg), len(vocabs)

In [ ]:
# create word embedding dictionary
embeddings = dict()

# zip words and their corresponding vectors
for i, (word,vector) in enumerate(zip(vocabs, vectors)):
    embeddings[word] = vector
    
# delete unused variables to free memory
del((i, word, vector))

In [ ]:
num_tokens = vocab_size
embedding_dim = 300
hits = 0
misses = 0


# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        
print("Converted %d words (%d misses)" % (hits, misses))

## Build Model

In [ ]:
model = Sequential()

In [ ]:
model.add(Input(shape=(maxlen, )))

In [ ]:
embed_size = 300
model.add(Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable=False))

In [ ]:
model.add(LSTM(60, return_sequences=True,name='lstm_layer'))

In [ ]:
model.add(GlobalMaxPool1D())

In [ ]:
model.add(Dropout(0.1))

In [ ]:
model.add(Dense(50, activation="relu"))

In [ ]:
model.add(Dropout(0.1))

In [ ]:
model.add(Dense(3, activation="softmax"))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
model.summary()

### Create plot for model architecture

In [ ]:
# Plot model
dot_img_file = 'model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file,
                          show_shapes=True,
                          show_layer_names=True,
                          rankdir="TB",
                          expand_nested=True,
                          dpi=96
                         )

### Train model

In [ ]:
# saving training history
csv_logger = CSVLogger('training.log', separator=',', append=False)

In [ ]:
batch_size = 16
epochs = 10
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, 
                    validation_data=(X_test, Y_test), callbacks=[csv_logger])

### Evaluate model

In [ ]:
score, accuracy = model.evaluate(X_test, Y_test, verbose = 1, batch_size = 32)
print("score: %.4f" % (score))
print("acc: %.4f" % (accuracy))

In [ ]:
# X_test

In [ ]:
y_pred_prob = model.predict(X_test, batch_size=32, verbose=1)

print("sample of probabilistic predictions: \n {}".format(y_pred_prob[:2]))

In [ ]:
roc_auc_score(Y_test, y_pred_prob, multi_class='ovr')

In [ ]:
# map classes to results
predictions = np.argmax(y_pred_prob, axis=1)
print("sample of y_pred: \n {}".format(predictions[:5]))

In [ ]:
precision = precision_score(Y_test.flatten(), predictions, average='weighted')
recall = recall_score(Y_test, predictions, average='weighted')
f1 = f1_score(Y_test, predictions, average='weighted')
accuracy = accuracy_score(Y_test, predictions, normalize=True)
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}, Accuracy: {:.4f}".format(precision, recall, f1, accuracy))

In [ ]:
# Load training history
history = pd.read_csv('training.log', sep=',', engine='python')

history.head()

## Plot Model Accuracy & Loss

In [ ]:
def plot_model(history):
    # plot history for accuracy
    plt.plot(history[ 'accuracy' ])
    plt.plot(history[ 'val_accuracy' ])
    plt.title( 'model accuracy' )
    plt.ylabel( 'accuracy' )
    plt.xlabel( 'epoch' )
    plt.legend([ 'train' , 'test' ], loc= 'lower right' )
    plt.show()

    # plot history for accuracy
    plt.plot(history[ 'loss' ])
    plt.plot(history[ 'val_loss' ])
    plt.title( 'model loss' )
    plt.ylabel( 'loss' )
    plt.xlabel( 'epoch' )
    plt.legend([ 'train' , 'test' ], loc= 'upper left' )
    plt.show()

## Confusion Matrix

In [ ]:
def get_rates(y_test, y_pred):
    cnf_matrix = confusion_matrix(y_test, np.argmax(y_pred, axis=1))

    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    
#     print(TP, FP, TN, FN)
#     print(TPR, TNR, PPV, NPV, FPR, FNR, FDR, ACC)
    
    return FPR, TPR

## Plot ROC Curve

In [ ]:
def plot_roc_curve(y_true, y_preds, clf):
    FPR, TPR = get_rates(y_true, y_preds)
    xx = np.arange(101) / float(100)
    AUC = roc_auc_score(y_true, y_preds, multi_class='ovr')
    AUC = metrics.auc(FPR, TPR)

    plt.xlim(0, 1.0)
    plt.ylim(0, 1.25)
    plt.plot([0.0, 0.0], [0.0, 1.0], color='green', linewidth=8)
    plt.plot([0.0, 1.0], [1.0, 1.0], color='green', label='Perfect Model', linewidth=4)
    plt.plot(xx,xx, linestyle='--', color='blue', label='Random Model')
    plt.plot(FPR, TPR, color='red', label='User Model')
    plt.title(clf+": ROC Curve - AUR value ="+str(AUC))
    plt.xlabel('% false positives')
    plt.ylabel('% true positives')
    plt.legend()
    plt.show()

In [ ]:
plot_model(history)

In [ ]:
plot_roc_curve(Y_test, y_pred_proba, "DNN")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_recall_fscore_support,classification_report

In [ ]:
classifier = LogisticRegression()
clf = "Logistic Regression"
classifier.fit(X_train, Y_train)
y_pred_proba = classifier.predict_proba(X=X_test)
plot_roc_curve(y_true=Y_test, y_preds=y_pred_proba, clf=clf)
y_pred = classifier.predict(X_test)
print('Classification report:\n',classification_report(Y_test,y_pred))

In [ ]:
classifier = RandomForestClassifier()
clf = "Random Forest"
classifier.fit(X_train, Y_train)
y_pred_proba = classifier.predict_proba(X=X_test)
plot_roc_curve(y_true=Y_test, y_preds=y_pred_proba, clf=clf)
y_pred = classifier.predict(X_test)
print('Classification report:\n',classification_report(Y_test,y_pred))


In [ ]:
classifier = MultinomialNB()
clf = "Multinomial Naive Bayes"
classifier.fit(X_train, Y_train)
y_pred_proba = classifier.predict_proba(X=X_test)
plot_roc_curve(y_true=Y_test, y_preds=y_pred_proba, clf=clf)
y_pred = classifier.predict(X_test)
print('Classification report:\n',classification_report(Y_test,y_pred))


In [ ]:

classifier = SGDClassifier(loss='log')
clf = "SGD"
classifier.fit(X_train, Y_train)
y_pred_proba = classifier.predict_proba(X=X_test)
plot_roc_curve(y_true=Y_test, y_preds=y_pred_proba, clf=clf)
y_pred = classifier.predict(X_test)
print('Classification report:\n',classification_report(Y_test,y_pred))


In [ ]:

classifier = KNeighborsClassifier()
clf = "KN"
classifier.fit(X_train, Y_train)
y_pred_proba = classifier.predict_proba(X=X_test)
plot_roc_curve(y_true=Y_test, y_preds=y_pred_proba, clf=clf)
y_pred = classifier.predict(X_test)
print('Classification report:\n',classification_report(Y_test,y_pred))


In [ ]:

classifier = DecisionTreeClassifier()
clf = "Decision Tree"
classifier.fit(X_train, Y_train)
y_pred_proba = classifier.predict_proba(X=X_test)
plot_roc_curve(y_true=Y_test, y_preds=y_pred_proba, clf=clf)
y_pred = classifier.predict(X_test)
print('Classification report:\n',classification_report(Y_test,y_pred))



In [ ]:
classifier = AdaBoostClassifier()
clf = "AdaBoost"
classifier.fit(X_train, Y_train)
y_pred_proba = classifier.predict_proba(X=X_test)
plot_roc_curve(y_true=Y_test, y_preds=y_pred_proba, clf=clf)
y_pred = classifier.predict(X_test)
print('Classification report:\n',classification_report(Y_test,y_pred))